## 가상현실RPG게임 라플라스의 악마는 날씨가 큰 변수를 차지하는 게임이다. 게임 밸런스 조정을 위해 한달 간의 날씨 데이터를 분석하려고 한다.

Laplace's Demon.csv

|컬럼|정의|type|
|:---|:---|:---:|
|date|날짜|object|
|area|지역명|object|
|temp|온도|float64|
|atemp|체감온도|float64|
|humidity|습도|float64|
|windspeed|풍속|float64|
|rain|날씨(값이 0이면 맑음, 1이면 비가 내림)|int64|  

정답 및 해설 : https://tjd229.tistory.com/24

In [96]:
import numpy as np
import pandas as pd

df = pd.read_csv('../content/tjd229/Laplace\'s Demon.csv')
df.shape

(992, 7)

### Q1. 체감온도(atemp)가 정규분포와의 관계를 분석하려 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 체감온도(atemp)가 0보다 큰 데이터를 Train Set으로, 체감온도(atemp)가 0이하인 데이터를 Test Set으로 분할한다.  
단계 2 : Train Set을 이용하여 LinearRegression을 학습하고, Test Set에 적용한다. 
- 독립 변수(총 4개) : 온도(temp), 습도(humidity), 풍속(windspeed), 날씨(rain)
- 종속 변수 : 체감온도(atemp)
        
단계 3 : Train Set에서 체감온도(atemp)의 평균과 표준편차를 구하고, 해당 평균과 표준편차를 가지는 정규 분포를 구한다.

단계 4 : 단계 2에서 구한 Test Set의 예측값 $ \hat{y_i} $와 단계 3에서 구한 정규 분포의 0.25 분위수에 해당하는 값 $ y $에 대하여 아래 정의된 Measure M을 계산한 값은?  
$$ M = \biggl(\frac{1}{n} \sum_{i=1}^{n}{(y - \hat{y_i})^2}\biggr)^{\frac{1}{2}}, \quad \hat{y_i}: 예측값, y: 실제값 $$


※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [97]:
from sklearn.linear_model import LinearRegression  
from scipy.stats import norm 
df1 = df.copy()

In [98]:
X_train = df1.loc[df1.atemp > 0, ['temp','humidity','windspeed','rain']]
X_test = df1.loc[df1.atemp <= 0, ['temp','humidity','windspeed','rain']]
y_train = df1.loc[df1.atemp > 0, 'atemp']
y_test = df1.loc[df1.atemp <= 0, 'atemp']

In [99]:
my_LR = LinearRegression()
my_LR.fit(X_train, y_train)
y_pred = my_LR.predict(X_test)

In [100]:
# 1-3 : y_train 의 정규분포

df1.loc[df1.atemp > 0,'atemp'].mean(), df1.loc[df1.atemp > 0,'atemp'].std()

(46.39257288379459, 34.36297082966668)

👉 norm.ppf  

norm : 정규분포 함수   norm(mean,std)   
binom : 이항분포 함수  binom(x,n,p)  
poissson : 포아송분포 함수 poisson(x,mu)  
expon : 지수분포 함수 expon(x,mu)  


pmf : 확률질량함수(probability mass function)  
pdf : 확률밀도함수(probability density function)  
cdf : 누적분포함수(cumulative distribution function)  
ppf : 누적분포함수의 역함수(inverse cumulative distribution function)  

https://datascienceschool.net/02%20mathematics/08.01%20%EC%82%AC%EC%9D%B4%ED%8C%8C%EC%9D%B4%EB%A5%BC%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20%ED%99%95%EB%A5%A0%EB%B6%84%ED%8F%AC%20%EB%B6%84%EC%84%9D.

In [101]:
# 정규분포 norm(mean, std) / my_norm.ppf(확률) = 기대값
my_norm= norm(df1.loc[df1.atemp > 0,'atemp'].mean(),df1.loc[df1.atemp > 0,'atemp'].std())

# df1.loc[df1.atemp > 0,'atemp'] 중 25% 에 해당되는 값
y_test2= my_norm.ppf(0.25) 
y_test2

23.21510127289747

In [102]:
my_norm.cdf(23.21510127289747)

0.25

In [103]:
df1.loc[df1.atemp > 0,'atemp'].quantile(0.25)

19.214476439079412

In [104]:
# RMSE 

(((y_pred - y_test2)**2).mean())**0.5

13.563655780091835

RMSE  
y_test 는 리스트 이지만, y_test2 정수 이므로,   
동일한 크기의 리스트에 y_test2 값을 모두 넣어야 RMSE 가 계산가능하다.

In [105]:
y_pred.shape

(322,)

In [106]:
y_test3 = y_test.copy()
y_test3[:] = y_test2
y_test3.shape

(322,)

In [107]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test3, y_pred, squared=False)


13.563655780091835

#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.linear_model import LinearRegression  
from scipy.stats import norm  
문제 지시 외 Default 값 사용  

### Q2. 지역별 강수량을 확인하려 한다.  다음 단계에 따라 분석을 수행하고 질문에 답하시오.


단계 1 : 날씨(rain) 컬럼에서 값이 0인 데이터를 -1로 바꾼다.

단계 2 : 32종류의 area별로, 아래의 단계 3을 수행하여, cum_precipitation 컬럼의 최대값과 최소값 차이가 가장 큰 값을 구하시오 

단계 3 : 각 area별로 날짜순으로 정렬 후, 아래 규칙에 따라 cum_precipitation 컬럼을 생성한다.  
- $z$를 cum_precipitation, $ x $를 rain이라 할 때, $ z_1 = x_1 $ 이고,  
- j>1에 대하여, $ z_j = \sum_{i=1}^{j}{x_i} $ 이다.  



In [108]:
df2 = df.copy()

In [109]:
df2['rain'] = df2.rain.replace(0,-1)

In [110]:
df2.head()

,date,area,temp,atemp,humidity,windspeed,rain
0,2023-01-13,Cerulean City,26.869606,-8.894446,43.138450,6.009597,1
1,2023-01-17,Cerulean City,18.385474,-4.988309,88.427747,20.864357,1
2,2023-01-31,Seafoam Islands,26.867670,0.659525,30.940357,30.733466,1
3,2023-01-31,Goldenrod City,24.329853,107.903405,64.554282,8.618055,-1
4,2023-01-23,Cianwood City,30.726903,75.236753,51.298311,13.717975,-1


In [111]:
areas = df2.area.unique()
areas

array(['Cerulean City', 'Seafoam Islands', 'Goldenrod City',
       'Cianwood City', 'Mt. Mortar', 'Olivine City', 'Viridian Forest',
       'Mt. Moon', 'Mahogany Town', 'Ruins of Alph', 'Whirl Islands',
       'Lavender Town', 'Mt. Silver', 'Tohjo Falls', 'Saffron City',
       'Cherrygrove City', 'Pallet Town', 'Pewter City',
       'Cinnabar Island', 'Power Plant', 'Vermillion City',
       'Blackthorn City', 'Azalea Town', 'Viridian City', 'Ecruteak City',
       'New Bark Town', 'Celadon City', 'Safari Zone', 'Fuchsia City',
       'Lake of Rage', 'Violet City', 'National Park'], dtype=object)

👉 groupby 대신 slicing (df2_0)

1. 특정 col 의 unique 한 값들의 리스트를 만든다. 
2. 해당 리스트 으로 for 구문을 돌린다. 
3. 공통된 unique 한 값(리스트값)을 갖는 데이터가 되도록 slicing 한다.
4. slicing 한 데이터 그룹에 필요한 계산을 한다. 
5. for 구문에 의해 다음 slicing 데이터 그룹 으로 이동한다. 

예) 

1. unique한 area 값으로 areas 리스트를 만든다. 
2. areas 리스트로 for 구문을 돌린다.
3. area 별로 slicing 한 데이터의 rain cumsun() 값을 계산한다.
4. 계산한 값을 slicing 한 데이터에 업데이트 한다. 
5. for 구문에 의해 다음 unique 한 area 값으로 slicing 된 데이터로 이동한다.

In [112]:
df2_0 = df2.copy()
MM_list=[]
max_cumsum = -999

for area_indx in areas:
    temp_df = df2_0.loc[df2_0.area == area_indx,:]                  # slicing df 에서 
    temp_list = temp_df.sort_values(by='date')['rain'].cumsum()     
    # date sort 한 rain의 cumsum 값 리스트 
    # reset_index 을 해도 처음 입력된 index 보다 큰 수가 있다면 입력이 안된다. 
    df2_0.loc[df2_0.area == area_indx,'result'] = temp_list.tolist()        
    # 을 원래 df2_0 의 result 에 입력, 입력할때는 인덱스가 없는 list 로 column 입력
    MM_list.append(temp_list.max()-temp_list.min())                 # 하고, 리스트 max() - min() 값 저장
    if max_cumsum < temp_list.max()-temp_list.min():                # 지금 slicing df 의 max-min 을 최고값과 비교한다.
        max_cumsum = temp_list.max()-temp_list.min()                # 최고값을 업데이트 한다.

max_cumsum

# for area_indx in areas:
#     sel_area = df2_0['area'] == area_indx
#     temp_result = df2_0.loc[sel_area,:].sort_values(by='date')['rain'].reset_index(drop='first').cumsum()
#     df2_0.loc[sel_area,'result'] = temp_result
#     MM_list.append(temp_result.max()-temp_result.min())
#     if max_cum < temp_result.max()-temp_result.min()
#         max_cum = temp_result.max()-temp_result.min()


18

In [113]:
# 각 area 별 max-min 값 보기 

pd.DataFrame( MM_list,
             index=areas,
             columns=['max-min'])

,max-min
Cerulean City,8
Seafoam Islands,6
Goldenrod City,6
Cianwood City,8
Mt. Mortar,11
Olivine City,9
Viridian Forest,9
Mt. Moon,10
Mahogany Town,6
Ruins of Alph,6


In [50]:
df2_0.loc[df.area == 'Lake of Rage','result'].max() - df2_0.loc[df.area == 'Lake of Rage','result'].min()

18.0

groupby 이용 (df2_1)

In [51]:
df2_1 = df2.groupby(['area','date'])['rain'].sum().reset_index().copy()
df2_1.head()

# groupby 로 인해 sort 된듯

,area,date,rain
0,Azalea Town,2023-01-01,-1
1,Azalea Town,2023-01-02,1
2,Azalea Town,2023-01-03,-1
3,Azalea Town,2023-01-04,-1
4,Azalea Town,2023-01-05,-1


In [64]:
# groupby 된 df2_1 을 이용하다.

cumsum_df = pd.DataFrame()
MM_list =[]

for area_indx in areas:
    cumsum_df[area_indx] = df2_1.loc[df2_1.area == area_indx,'rain'].reset_index(drop=True).cumsum()
    MM_list.append(cumsum_df[area_indx].max() - cumsum_df[area_indx].min())


In [67]:
cumsum_df.head()

,Cerulean City,Seafoam Islands,Goldenrod City,Cianwood City,Mt. Mortar,Olivine City,Viridian Forest,Mt. Moon,Mahogany Town,Ruins of Alph,...,Azalea Town,Viridian City,Ecruteak City,New Bark Town,Celadon City,Safari Zone,Fuchsia City,Lake of Rage,Violet City,National Park
0,-1,-1,1,1,1,-1,-1,1,-1,1,...,-1,1,1,-1,1,1,1,1,-1,1
1,0,0,2,2,0,-2,0,2,-2,0,...,0,2,2,0,0,2,0,2,0,0
2,1,-1,1,1,-1,-1,-1,3,-1,1,...,-1,1,1,-1,-1,3,-1,3,-1,-1
3,0,-2,2,2,-2,0,0,4,0,2,...,-2,2,2,-2,-2,2,0,2,0,-2
4,1,-3,3,1,-1,1,1,5,-1,3,...,-3,3,1,-1,-1,1,-1,3,-1,-3


In [66]:

pd.DataFrame( MM_list,
            index=areas,
            columns=['max_min'])

,max_min
Cerulean City,8
Seafoam Islands,6
Goldenrod City,6
Cianwood City,8
Mt. Mortar,11
Olivine City,9
Viridian Forest,9
Mt. Moon,10
Mahogany Town,6
Ruins of Alph,6


In [23]:

result_list=[]

for area_indx in areas:
    temp_df= df2_1.loc[df2_1.area == area_indx,].copy()
    temp_df['cum_precipitation'] = temp_df['rain'].cumsum()
    delta = temp_df['cum_precipitation'].max()-temp_df['cum_precipitation'].min()
    result_list.append([area_indx,delta])

result_df = pd.DataFrame(result_list,
                         columns=['Area','max_min'])

result_df



,Area,max_min
0,Cerulean City,8
1,Seafoam Islands,6
2,Goldenrod City,6
3,Cianwood City,8
4,Mt. Mortar,11
5,Olivine City,9
6,Viridian Forest,9
7,Mt. Moon,10
8,Mahogany Town,6
9,Ruins of Alph,6


cumsum()  

뭔가 apply, itertuples 로 가능할것 같은데.  
apply, itertuples 는 한 row 씩 계산하는 거라. cumsum 을 만들어야 함.


In [86]:
df2_2 = df2.copy()

df2_1.loc[df2_1.area == area_indx,]

### Q3. 날씨에 따른 게임 밸런스를 확인하기 위해 군집화를 하려 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.


단계 1 : 온도, 체감온도, 습도, 풍속을 Z-score 표준화(Standardization) 한다.  
단계 2 : 독립 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2개로 한다.  
- 독립 변수(총 5개) : 온도(temp), 체감온도(atemp), 습도(humidity), 풍속(windspeed), 날씨(rain)  

단계 3 : 군집 별 체감온도(atemp) 평균을 구하고 두 평균을 모두 더한 값을 구하시오  
            
※ 정답은 내림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [24]:
from sklearn.cluster import KMeans  
from sklearn.preprocessing import StandardScaler  

df3 = df.copy()

df3.head()

,date,area,temp,atemp,humidity,windspeed,rain
0,2023-01-13,Cerulean City,26.869606,-8.894446,43.138450,6.009597,1
1,2023-01-17,Cerulean City,18.385474,-4.988309,88.427747,20.864357,1
2,2023-01-31,Seafoam Islands,26.867670,0.659525,30.940357,30.733466,1
3,2023-01-31,Goldenrod City,24.329853,107.903405,64.554282,8.618055,0
4,2023-01-23,Cianwood City,30.726903,75.236753,51.298311,13.717975,0


#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.cluster import KMeans  
from sklearn.preprocessing import StandardScaler  
random_state=229  
문제 지시 외 Default 값 사용  

StandardScaler()

In [25]:


m_SDS = StandardScaler()
df3[['temp','atemp','humidity','windspeed']] = m_SDS.fit_transform(df3[['temp','atemp','humidity','windspeed']])

df3[['temp','atemp','humidity','windspeed']]

,temp,atemp,humidity,windspeed
0,0.524821,-0.604300,-1.250768,-0.859196
1,-0.895741,-0.524980,1.413644,0.740058
2,0.524496,-0.410293,-1.968393,1.802559
3,0.099571,1.767447,0.009146,-0.578371
4,1.170677,1.104104,-0.770716,-0.029317
...,...,...,...,...
987,-0.894014,0.440263,0.177634,0.616921
988,-0.561965,0.688070,-1.487709,1.607133
989,-0.388304,-1.053913,-1.063798,-2.525033
990,0.523699,-2.402308,-0.156465,0.393403


KMean

In [26]:
df4 = df.copy()
X_train = df3[['temp','atemp','humidity','windspeed','rain']]

my_km = KMeans(n_clusters=2,init='k-means++',n_init=10,random_state=229)
my_km.fit(X_train)
df4['culster'] = my_km.labels_
df4.head()

,date,area,temp,atemp,humidity,windspeed,rain,culster
0,2023-01-13,Cerulean City,26.869606,-8.894446,43.138450,6.009597,1,1
1,2023-01-17,Cerulean City,18.385474,-4.988309,88.427747,20.864357,1,1
2,2023-01-31,Seafoam Islands,26.867670,0.659525,30.940357,30.733466,1,1
3,2023-01-31,Goldenrod City,24.329853,107.903405,64.554282,8.618055,0,0
4,2023-01-23,Cianwood City,30.726903,75.236753,51.298311,13.717975,0,0


In [27]:
answer = df4.loc[df4.culster ==1,'atemp'].mean()+df4.loc[df4.culster ==0,'atemp'].mean()
int(answer*100)/100

50.89